In [12]:
# import library
import os
from scipy import *
import scipy.ndimage.filters
import scipy.io
import scipy.stats
import scipy.linalg as linalg
import numpy as np
import math
import pickle
import pandas as pd
from scipy.optimize import minimize
import time

def data_loader_h5(mat_file): #, imu_file=None, sync_file=None
    if not os.path.exists(mat_file):
        raise Exception('mat file: %s does not exist !!! Please check the given path.' % mat_file)
    mat_data = pd.read_hdf(mat_file, key='df') #adapting to mir's h5 file
    
    mat_data = mat_data.reset_index()
    # time = df_new['timestamp_ms_mini']

    return mat_data



rec_path = '/data/big_rim/rsync_dcc_sum/Oct3V1/2024_10_25/20241002PMCr2_17_05'

hdf5_file_path = os.path.join(rec_path, 'MIR_Aligned/aligned_predictions_with_ca_and_dF_F.h5')

data_load = data_loader_h5(hdf5_file_path)

In [32]:
import numpy as np
import pandas as pd

def get_body_rotations(sorted_point_data):
    print('Processing to get body related rotation matrix ...... ')
    nf = len(sorted_point_data)  # number of frames

    r_roots = np.zeros((nf, 3, 3))  #
    r_root_inv = np.zeros((nf, 3, 3))  #
    r_root_inv_oriented = np.zeros((nf, 3, 3))  #
    dir_backs = np.zeros((nf, 3))  #
    r_roots[:] = np.nan
    r_root_inv[:] = np.nan
    r_root_inv_oriented[:] = np.nan
    dir_backs[:] = np.nan

    timeofheadnans = []
    # DO IT ALL FOR THE NEW ROOT NODE (ASSUMING BUTT TO NECK!)
    for t in np.arange(nf):
        if np.isnan(sorted_point_data[t, 4, 0]) or np.isnan(sorted_point_data[t, 6, 0]):  # 4 is neck and 6 is tail
            continue
    
        # THIS IS A STUPID WAY OF CREATING RROOTS (ANGLE OF BODY RELATIVE TO A WALL)
        # It should just be xdir with z=0...
        xdir = sorted_point_data[t, 4, :] - sorted_point_data[t, 6, :]  # from tail to neck
        xdir[2] = 0.
        ll = np.linalg.norm(xdir)
        if ll < 0.001:
            continue
        xdir = xdir / ll  # x direction of the animal from the top view (chosen)
        zdir = np.ravel(np.array([0, 0, 1]))
        ydir = np.cross(zdir, xdir)  # y direction according to right-hand rule
        shitinv = (np.array([xdir, ydir, zdir])) + 0.  # mapping from global coordinates to animal coordinates
        r_roots[t, :, :] = np.transpose(shitinv)  # mapping from animal to global from the top view
    
        # HERE IS THE REAL ROOT ANGLE!!!
        # SAME AS WITHLOCKINZ = TRUE AND WITHROTATIONS = TRUE
        xdir = sorted_point_data[t, 4, :] - sorted_point_data[t, 6, :]
        ll = np.linalg.norm(xdir)
        if ll < 0.001:
            continue
        xdir = xdir / ll  # unit vector from tail to neck in global coordinates (not top view)
        ydir = np.zeros(3)
        ydir[0] = -xdir[1] + 0.
        ydir[1] = xdir[0] + 0.
        ydir[2] = 0.
        ydir = ydir / np.linalg.norm(ydir)  # an orthogonal vector to the x dir in xy plane
        zdir = np.cross(xdir, ydir)
        zdir = zdir / np.linalg.norm(zdir)
        r_root_inv[t, :, :] = (np.array([xdir, ydir, zdir])) + 0.  # for ego3
    
        # HERE IS THE OTHER OTHER OTHER ROOT ANGLE!!!
        xdir = sorted_point_data[t, 4, :] - sorted_point_data[t, 6, :]
        xdir[2] = 0  # so we just keep the planar direction from the body coordinates!!!
        ll = np.linalg.norm(xdir)
        if ll < 0.001:
            timeofheadnans.append(t)
            continue
        xdir = xdir / ll
        ydir = np.zeros(3)
        ydir[0] = -xdir[1] + 0.
        ydir[1] = xdir[0] + 0.
        ydir[2] = 0.
        ydir = ydir / np.linalg.norm(ydir)
        zdir = np.zeros(3)
        zdir[2] = 1.
        r_root_inv_oriented[t, :, :] = (np.array([xdir, ydir, zdir])) + 0.  # for ego2
    
        # if( WITHLOCKEDINZ == False and WITHROTATIONS == True ):
        # xdir = sorted_point_data[t,4,:] - sorted_point_data[t,6,:]
        # ll = np.linalg.norm(xdir)
        # if(ll<0.001):
        # continue
        # xdir[2] = 0.
        # xdir = xdir / ll
        # ydir = zeros(3)
        # ydir[0] = -xdir[1]+0.
        # ydir[1] = xdir[0]+0.
        # ydir[2] = 0.
        # ydir = ydir/np.linalg.norm(ydir)
        # zdir = zeros(3)
        # zdir[2] = 1.
        # RrootINV = ( np.array([xdir, ydir, zdir]) ) + 0.
        if ~np.isnan(sorted_point_data[t, 6, 0]):
            dir_to_butt = sorted_point_data[t, 4, :] - sorted_point_data[t, 5, :]
            dir_to_butt = dir_to_butt / np.linalg.norm(dir_to_butt)
            dir_to_butt = np.dot(r_root_inv[t, :, :], dir_to_butt)
            dir_backs[t, :] = dir_to_butt + 0.
    
    return r_roots, r_root_inv_oriented, r_root_inv, dir_backs


def get_body_rotations_df(df, spineF_kp=4, tailB_kp=6, spineM_kp=5):
    print('Processing to get body related rotation matrix ......')
    
    nf = df.shape[0]
    r_roots = np.zeros((nf, 3, 3))  
    r_root_inv = np.zeros((nf, 3, 3))  
    r_root_inv_oriented = np.zeros((nf, 3, 3))  
    dir_backs = np.zeros((nf, 3))  

    r_roots[:] = np.nan
    r_root_inv[:] = np.nan
    r_root_inv_oriented[:] = np.nan
    dir_backs[:] = np.nan

    
    timeofheadnans = []

    # THIS IS A STUPID WAY OF CREATING RROOTS (ANGLE OF BODY RELATIVE TO A WALL)
        # It should just be xdir with z=0...
    # Extract x, y, and z coordinates for relevant keypoints
    spineF_x, spineF_y, spineF_z = df[f"kp{spineF_kp}_x"], df[f"kp{spineF_kp}_y"], df[f"kp{spineF_kp}_z"]
    tailB_x, tailB_y, tailB_z = df[f"kp{tailB_kp}_x"], df[f"kp{tailB_kp}_y"], df[f"kp{tailB_kp}_z"]
    spineM_x, spineM_y, spineM_z = df[f"kp{spineM_kp}_x"], df[f"kp{spineM_kp}_y"], df[f"kp{spineM_kp}_z"]

    # Precompute valid mask
    valid_mask = ~np.isnan(spineF_x) & ~np.isnan(tailB_x) #the orig script only cheks for x, whatever...
    
    # Compute x-direction (from tail to neck)
    xdir = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, np.zeros_like(spineF_z)))  # Z-projection
    ll = np.linalg.norm(xdir, axis=1, keepdims=True)
    valid_mask &= (ll[:, 0] >= 0.001)
    xdir[~valid_mask] = np.nan
    xdir[valid_mask] /= ll[valid_mask]  # Normalize
    
    # Compute y-direction using right-hand rule
    zdir = np.array([0, 0, 1])  # Fixed vertical axis
    ydir = np.cross(zdir, xdir)
   

    # Construct the transformation matrix for body-to-global mapping
    r_roots = np.stack([xdir, ydir, np.tile(zdir, (nf, 1))], axis=1)  # (nf, 3, 3)
    r_roots = np.transpose(r_roots, (0, 2, 1))  # Transpose for correct mapping


    # HERE IS THE REAL ROOT ANGLE!!!
        # SAME AS WITHLOCKINZ = TRUE AND WITHROTATIONS = TRUE
    # Compute actual body rotation without locking Z
    xdir_no_proj = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, spineF_z - tailB_z))
    ll_no_proj = np.linalg.norm(xdir_no_proj, axis=1, keepdims=True)
    valid_mask_no_proj = (ll_no_proj[:, 0] >= 0.001)
    xdir_no_proj[~valid_mask_no_proj] = np.nan
    xdir_no_proj[valid_mask_no_proj] /= ll_no_proj[valid_mask_no_proj]

    ydir_no_proj = np.column_stack((-xdir_no_proj[:, 1], xdir_no_proj[:, 0], np.zeros(nf)))
    ydir_no_proj /= np.linalg.norm(ydir_no_proj, axis=1, keepdims=True)
    
    zdir_no_proj = np.cross(xdir_no_proj, ydir_no_proj)

    zdir_no_proj /= np.linalg.norm(zdir_no_proj, axis=1, keepdims=True)
    r_root_inv = np.stack([xdir_no_proj, ydir_no_proj, zdir_no_proj], axis=1)

 # HERE IS THE OTHER OTHER OTHER ROOT ANGLE!!!
    # Compute 2D orientation (planar body direction)
    xdir_planar = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, np.zeros(nf)))
    ll_planar = np.linalg.norm(xdir_planar, axis=1, keepdims=True)
    valid_mask_planar = (ll_planar[:, 0] >= 0.001)
    xdir_planar[~valid_mask_planar] = np.nan
    xdir_planar[valid_mask_planar] /= ll_planar[valid_mask_planar]

    ydir_planar = np.column_stack((-xdir_planar[:, 1], xdir_planar[:, 0], np.zeros(nf)))
    ydir_planar /= np.linalg.norm(ydir_planar, axis=1, keepdims=True)

    zdir_fixed = np.tile([0, 0, 1], (nf, 1))

    r_root_inv_oriented = np.stack([xdir_planar, ydir_planar, zdir_fixed], axis=1)

    # Compute direction to mid-spine (if available)
    valid_mid_spine = ~np.isnan(spineM_x)
    if np.any(valid_mid_spine):
        dir_to_butt = np.column_stack((spineF_x - spineM_x, spineF_y - spineM_y, spineF_z - spineM_z))
        ll_butt = np.linalg.norm(dir_to_butt, axis=1, keepdims=True)
        valid_butt_mask = (ll_butt[:, 0] >= 0.001)
        dir_to_butt[~valid_butt_mask] = np.nan
        dir_to_butt[valid_butt_mask] /= ll_butt[valid_butt_mask]

        dir_to_butt = np.einsum('ijk,ik->ij', r_root_inv, dir_to_butt)
        dir_backs = dir_to_butt

    return r_roots, r_root_inv_oriented, r_root_inv, dir_backs

# Generate test data (or load real data if available)
nf = 100  # Number of frames
num_keypoints = 8  # Adjust this to match the actual number of keypoints

# Simulate random sorted_point_data for the original function
np.random.seed(0)
sorted_point_data = np.random.randn(nf, num_keypoints, 3)  # (frames, keypoints, coordinates)
sorted_point_data[::10, 4:, :] = np.nan  # Add some NaNs to simulate missing data

# Convert sorted_point_data to DataFrame format for the revised function
columns = [f"kp{i}_{axis}" for i in range(num_keypoints) for axis in ['x', 'y', 'z']]
data_flattened = sorted_point_data.reshape(nf, -1)
df_test = pd.DataFrame(data_flattened, columns=columns)

# Run both functions
r_roots_orig, r_root_inv_oriented_orig, r_root_inv_orig, dir_backs_orig = get_body_rotations(sorted_point_data)
r_roots_rev, r_root_inv_oriented_rev, r_root_inv_rev, dir_backs_rev = get_body_rotations_df(df_test)

# Compare outputs
def compare_matrices(mat1, mat2, tolerance=1e-6):
    """Compare two matrices for numerical equality within a tolerance."""
    return np.allclose(mat1, mat2, atol=tolerance, equal_nan=True)

# Check each output
results = {
    "r_roots": compare_matrices(r_roots_orig, r_roots_rev),
    "r_root_inv_oriented": compare_matrices(r_root_inv_oriented_orig, r_root_inv_oriented_rev),
    "r_root_inv": compare_matrices(r_root_inv_orig, r_root_inv_rev),
    "dir_backs": compare_matrices(dir_backs_orig, dir_backs_rev),
}

print("Validation Results:", results)


Processing to get body related rotation matrix ...... 
Processing to get body related rotation matrix ......
Validation Results: {'r_roots': False, 'r_root_inv_oriented': False, 'r_root_inv': True, 'dir_backs': True}


In [38]:
def get_body_rotations_df_new(df, spineF_kp=4, tailB_kp=6, spineM_kp=5):
    print('Processing to get body related rotation matrix ......')
    
    nf = df.shape[0]
    r_roots = np.zeros((nf, 3, 3))  
    r_root_inv = np.zeros((nf, 3, 3))  
    r_root_inv_oriented = np.zeros((nf, 3, 3))  
    dir_backs = np.zeros((nf, 3))  

    r_roots[:] = np.nan
    r_root_inv[:] = np.nan
    r_root_inv_oriented[:] = np.nan
    dir_backs[:] = np.nan

    # Extract x, y, and z coordinates for relevant keypoints
    spineF_x, spineF_y, spineF_z = df[f"kp{spineF_kp}_x"], df[f"kp{spineF_kp}_y"], df[f"kp{spineF_kp}_z"]
    tailB_x, tailB_y, tailB_z = df[f"kp{tailB_kp}_x"], df[f"kp{tailB_kp}_y"], df[f"kp{tailB_kp}_z"]
    spineM_x, spineM_y, spineM_z = df[f"kp{spineM_kp}_x"], df[f"kp{spineM_kp}_y"], df[f"kp{spineM_kp}_z"]

    # Precompute valid mask
    valid_mask = ~np.isnan(spineF_x) & ~np.isnan(tailB_x) 

    # 1. NaN handling when computing xdir (body direction from tail to neck)
    xdir = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, np.zeros_like(spineF_z)))
    xdir[~valid_mask] = np.nan  # Explicitly set NaN where mask is invalid
    ll = np.linalg.norm(xdir, axis=1, keepdims=True)
    valid_mask &= (ll[:, 0] >= 0.001)
    xdir[~valid_mask] = np.nan  # Propagate NaNs after length check
    xdir[valid_mask] /= ll[valid_mask]

    # Compute y-direction using right-hand rule
    zdir = np.array([0, 0, 1])
    ydir = np.cross(zdir, xdir)
    ydir[~valid_mask] = np.nan  # Ensure ydir is NaN where xdir is NaN

    # Construct the transformation matrix for body-to-global mapping
    r_roots = np.stack([xdir, ydir, np.tile(zdir, (nf, 1))], axis=1)
    r_roots[~valid_mask] = np.nan  # Set entire matrix to NaN for invalid frames
    r_roots = np.transpose(r_roots, (0, 2, 1))

    # 2. NaN handling for non-projected body rotation
    xdir_no_proj = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, spineF_z - tailB_z))
    xdir_no_proj[~valid_mask] = np.nan  # Set NaN where invalid
    ll_no_proj = np.linalg.norm(xdir_no_proj, axis=1, keepdims=True)
    valid_mask_no_proj = (ll_no_proj[:, 0] >= 0.001)
    xdir_no_proj[~valid_mask_no_proj] = np.nan  # Propagate NaN if length is too short
    xdir_no_proj[valid_mask_no_proj] /= ll_no_proj[valid_mask_no_proj]

    ydir_no_proj = np.column_stack((-xdir_no_proj[:, 1], xdir_no_proj[:, 0], np.zeros(nf)))
    ydir_no_proj[~valid_mask_no_proj] = np.nan  # Ensure NaN in ydir
    ydir_no_proj /= np.linalg.norm(ydir_no_proj, axis=1, keepdims=True)

    zdir_no_proj = np.cross(xdir_no_proj, ydir_no_proj)
    zdir_no_proj[~valid_mask_no_proj] = np.nan  # Ensure zdir gets NaNs
    zdir_no_proj /= np.linalg.norm(zdir_no_proj, axis=1, keepdims=True)
    r_root_inv = np.stack([xdir_no_proj, ydir_no_proj, zdir_no_proj], axis=1)
    r_root_inv[~valid_mask_no_proj] = np.nan

    # 3. NaN handling for planar body direction
    xdir_planar = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, np.zeros(nf)))
    xdir_planar[~valid_mask] = np.nan
    ll_planar = np.linalg.norm(xdir_planar, axis=1, keepdims=True)
    valid_mask_planar = (ll_planar[:, 0] >= 0.001)
    xdir_planar[~valid_mask_planar] = np.nan  # Propagate NaNs where length is too short
    xdir_planar[valid_mask_planar] /= ll_planar[valid_mask_planar]

    ydir_planar = np.column_stack((-xdir_planar[:, 1], xdir_planar[:, 0], np.zeros(nf)))
    ydir_planar[~valid_mask_planar] = np.nan
    ydir_planar /= np.linalg.norm(ydir_planar, axis=1, keepdims=True)

    zdir_fixed = np.tile([0, 0, 1], (nf, 1))
    r_root_inv_oriented = np.stack([xdir_planar, ydir_planar, zdir_fixed], axis=1)
    r_root_inv_oriented[~valid_mask_planar] = np.nan

    # 4. NaN handling for direction to mid-spine
    valid_mid_spine = ~np.isnan(spineM_x)
    dir_to_butt = np.column_stack((spineF_x - spineM_x, spineF_y - spineM_y, spineF_z - spineM_z))
    dir_to_butt[~valid_mid_spine] = np.nan
    ll_butt = np.linalg.norm(dir_to_butt, axis=1, keepdims=True)
    valid_butt_mask = (ll_butt[:, 0] >= 0.001)
    dir_to_butt[~valid_butt_mask] = np.nan  # Propagate NaNs if length is too short
    dir_to_butt[valid_butt_mask] /= ll_butt[valid_butt_mask]

    dir_to_butt = np.einsum('ijk,ik->ij', r_root_inv, dir_to_butt)
    dir_backs = dir_to_butt

    return r_roots, r_root_inv_oriented, r_root_inv, dir_backs


In [39]:
import numpy as np
import pandas as pd

# Generate test data (or load real data if available)
nf = 100  # Number of frames
num_keypoints = 8  # Adjust this to match the actual number of keypoints

# Simulate random sorted_point_data for the original function
np.random.seed(0)
sorted_point_data = np.random.randn(nf, num_keypoints, 3)  # (frames, keypoints, coordinates)
sorted_point_data[::10, 4:, :] = np.nan  # Add some NaNs to simulate missing data

# Convert sorted_point_data to DataFrame format for the revised function
columns = [f"kp{i}_{axis}" for i in range(num_keypoints) for axis in ['x', 'y', 'z']]
data_flattened = sorted_point_data.reshape(nf, -1)
df_test = pd.DataFrame(data_flattened, columns=columns)

# Run both functions
r_roots_orig, r_root_inv_oriented_orig, r_root_inv_orig, dir_backs_orig = get_body_rotations(sorted_point_data)
r_roots_rev, r_root_inv_oriented_rev, r_root_inv_rev, dir_backs_rev = get_body_rotations_df_new(df_test)

# Comparison function with debug info
def compare_and_debug(name, mat1, mat2, tolerance=1e-6):
    """Compare two matrices and print debug info."""
    if mat1.shape != mat2.shape:
        print(f"Shape mismatch for {name}: {mat1.shape} vs {mat2.shape}")
        return False

    # Check element-wise differences
    diff = np.abs(mat1 - mat2)
    max_diff = np.nanmax(diff)
    max_diff_idx = np.unravel_index(np.nanargmax(diff), diff.shape)

    # Check if within tolerance
    within_tolerance = np.allclose(mat1, mat2, atol=tolerance, equal_nan=True)
    
    if not within_tolerance:
        print(f"Mismatch found in {name}:")
        print(f"  Max absolute difference: {max_diff} at index {max_diff_idx}")
        print(f"  Value in original: {mat1[max_diff_idx]}")
        print(f"  Value in revised: {mat2[max_diff_idx]}")
        print(f"  Difference array (first 5 non-zero diffs):")
        nonzero_diff_indices = np.argwhere(~np.isclose(mat1, mat2, atol=tolerance, equal_nan=True))
        for idx in nonzero_diff_indices[:5]:
            idx_tuple = tuple(idx)
            print(f"    Index: {idx_tuple}, Original: {mat1[idx_tuple]}, Revised: {mat2[idx_tuple]}, Diff: {diff[idx_tuple]}")
    else:
        print(f"{name} matches within tolerance.")
    
    return within_tolerance

# Run comparisons with debugging
results = {
    "r_roots": compare_and_debug("r_roots", r_roots_orig, r_roots_rev),
    "r_root_inv_oriented": compare_and_debug("r_root_inv_oriented", r_root_inv_oriented_orig, r_root_inv_oriented_rev),
    "r_root_inv": compare_and_debug("r_root_inv", r_root_inv_orig, r_root_inv_rev),
    "dir_backs": compare_and_debug("dir_backs", dir_backs_orig, dir_backs_rev),
}

print("\nFinal Validation Results:", results)


Processing to get body related rotation matrix ...... 
Processing to get body related rotation matrix ......
r_roots matches within tolerance.
r_root_inv_oriented matches within tolerance.
r_root_inv matches within tolerance.
dir_backs matches within tolerance.

Final Validation Results: {'r_roots': True, 'r_root_inv_oriented': True, 'r_root_inv': True, 'dir_backs': True}


In [36]:
import numpy as np
import pandas as pd

# Simulated sorted_point_data (original format)
np.random.seed(0)
nf = 100  # Number of frames
num_keypoints = 8  # Number of keypoints in each frame
sorted_point_data = np.random.randn(nf, num_keypoints, 3)  # Shape: (frames, keypoints, 3)
sorted_point_data[::10, 4:, :] = np.nan  # Add some NaNs to simulate missing data

# Step 1: Convert sorted_point_data to DataFrame format
columns = [f"kp{i}_{axis}" for i in range(num_keypoints) for axis in ['x', 'y', 'z']]
data_flattened = sorted_point_data.reshape(nf, -1)  # Flatten sorted_point_data to (nf, num_keypoints * 3)
df_test = pd.DataFrame(data_flattened, columns=columns)

# Step 2: Convert df_test back to sorted_point_data format
reconstructed_data = df_test.values.reshape(nf, num_keypoints, 3)

# Step 3: Compare original and reconstructed data
def compare_data(arr1, arr2, tolerance=1e-6):
    """Compare two arrays for equality within a tolerance, including NaN values."""
    return np.allclose(arr1, arr2, atol=tolerance, equal_nan=True)

# Perform the comparison
comparison_result = compare_data(sorted_point_data, reconstructed_data)

# Output the result
print("Is df_test correctly representing sorted_point_data?", comparison_result)


Is df_test correctly representing sorted_point_data? True


In [35]:
df_test

,kp0_x,kp0_y,kp0_z,kp1_x,kp1_y,kp1_z,kp2_x,kp2_y,kp2_z,kp3_x,...,kp4_z,kp5_x,kp5_y,kp5_z,kp6_x,kp6_y,kp6_z,kp7_x,kp7_y,kp7_z
0,1.764052,0.400157,0.978738,2.240893,1.867558,-0.977278,0.950088,-0.151357,-0.103219,0.410599,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.269755,-1.454366,0.045759,-0.187184,1.532779,1.469359,0.154947,0.378163,-0.887786,-1.980796,...,-0.387327,-0.302303,-1.048553,-1.420018,-1.706270,1.950775,-0.509652,-0.438074,-1.252795,0.777490
2,-1.613898,-0.212740,-0.895467,0.386902,-0.510805,-1.180632,-0.028182,0.428332,0.066517,0.302472,...,-0.813146,-1.726283,0.177426,-0.401781,-1.630198,0.462782,-0.907298,0.051945,0.729091,0.128983
3,1.139401,-1.234826,0.402342,-0.684810,-0.870797,-0.578850,-0.311553,0.056165,-1.165150,0.900826,...,1.178780,-0.179925,-1.070753,1.054452,-0.403177,1.222445,0.208275,0.976639,0.356366,0.706573
4,0.010500,1.785870,0.126912,0.401989,1.883151,-1.347759,-1.270485,0.969397,-1.173123,1.943621,...,1.867559,0.906045,-0.861226,1.910065,-0.268003,0.802456,0.947252,-0.155010,0.614079,0.922207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.093895,0.205797,-1.306522,-0.973376,0.239087,-0.607887,-0.933316,-0.034475,0.072678,-0.205834,...,2.464322,0.193832,1.132005,-0.560981,-1.362941,-0.791757,-0.268010,-0.496608,1.336386,-0.120041
96,0.461469,-0.046481,-0.433554,0.037996,1.714051,-0.767949,0.766990,-1.026007,-0.459626,0.003583,...,0.650042,-0.364170,0.238682,-0.116222,-1.943457,0.508299,0.583368,0.926605,1.800463,-1.195104
97,0.516507,0.409295,-0.419082,0.397106,0.499647,-1.218684,0.246223,-0.917984,-0.651856,-1.774745,...,-1.542288,0.862148,-0.118587,0.488371,0.965936,1.422605,1.961227,-0.072239,0.311124,-1.078361
98,1.061600,-1.184887,-1.805252,0.830386,-0.521697,0.777607,0.408075,-1.630003,-2.719679,-1.096602,...,0.169878,0.090826,-0.481393,1.397065,1.497715,0.565267,-1.799771,-1.104690,0.407130,-0.628558


In [34]:
sorted_point_data.shape

(100, 8, 3)

In [28]:
data_load.shape

(8894, 245)

In [ ]:
def get_body_rotations(df, spineF_kp=4, tailB_kp=6, spineM_kp=5):
    print('Processing to get body related rotation matrix ......')
    
    nf = df.shape[0]
    r_roots = np.zeros((nf, 3, 3))  
    r_root_inv = np.zeros((nf, 3, 3))  
    r_root_inv_oriented = np.zeros((nf, 3, 3))  
    dir_backs = np.zeros((nf, 3))  

    r_roots[:] = np.nan
    r_root_inv[:] = np.nan
    r_root_inv_oriented[:] = np.nan
    dir_backs[:] = np.nan

    
    timeofheadnans = []

    # THIS IS A STUPID WAY OF CREATING RROOTS (ANGLE OF BODY RELATIVE TO A WALL)
        # It should just be xdir with z=0...
    # Extract x, y, and z coordinates for relevant keypoints
    spineF_x, spineF_y, spineF_z = df[f"kp{spineF_kp}_x"], df[f"kp{spineF_kp}_y"], df[f"kp{spineF_kp}_z"]
    tailB_x, tailB_y, tailB_z = df[f"kp{tailB_kp}_x"], df[f"kp{tailB_kp}_y"], df[f"kp{tailB_kp}_z"]
    spineM_x, spineM_y, spineM_z = df[f"kp{spineM_kp}_x"], df[f"kp{spineM_kp}_y"], df[f"kp{spineM_kp}_z"]

    # Precompute valid mask
    valid_mask = ~np.isnan(spineF_x) & ~np.isnan(tailB_x) #the orig script only cheks for x, whatever...
    
    # Compute x-direction (from tail to neck)
    xdir = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, np.zeros_like(spineF_z)))  # Z-projection
    ll = np.linalg.norm(xdir, axis=1, keepdims=True)
    valid_mask &= (ll[:, 0] >= 0.001)
    xdir[~valid_mask] = np.nan
    xdir[valid_mask] /= ll[valid_mask]  # Normalize
    
    # Compute y-direction using right-hand rule
    zdir = np.array([0, 0, 1])  # Fixed vertical axis
    ydir = np.cross(zdir, xdir)
   

    # Construct the transformation matrix for body-to-global mapping
    r_roots = np.stack([xdir, ydir, np.tile(zdir, (nf, 1))], axis=1)  # (nf, 3, 3)
    r_roots = np.transpose(r_roots, (0, 2, 1))  # Transpose for correct mapping


    # HERE IS THE REAL ROOT ANGLE!!!
        # SAME AS WITHLOCKINZ = TRUE AND WITHROTATIONS = TRUE
    # Compute actual body rotation without locking Z
    xdir_no_proj = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, spineF_z - tailB_z))
    ll_no_proj = np.linalg.norm(xdir_no_proj, axis=1, keepdims=True)
    valid_mask_no_proj = (ll_no_proj[:, 0] >= 0.001)
    xdir_no_proj[~valid_mask_no_proj] = np.nan
    xdir_no_proj[valid_mask_no_proj] /= ll_no_proj[valid_mask_no_proj]

    ydir_no_proj = np.column_stack((-xdir_no_proj[:, 1], xdir_no_proj[:, 0], np.zeros(nf)))
    ydir_no_proj /= np.linalg.norm(ydir_no_proj, axis=1, keepdims=True)
    
    zdir_no_proj = np.cross(xdir_no_proj, ydir_no_proj)

    zdir_no_proj /= np.linalg.norm(zdir_no_proj, axis=1, keepdims=True)
    r_root_inv = np.stack([xdir_no_proj, ydir_no_proj, zdir_no_proj], axis=1)

 # HERE IS THE OTHER OTHER OTHER ROOT ANGLE!!!
    # Compute 2D orientation (planar body direction)
    xdir_planar = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, np.zeros(nf)))
    ll_planar = np.linalg.norm(xdir_planar, axis=1, keepdims=True)
    valid_mask_planar = (ll_planar[:, 0] >= 0.001)
    xdir_planar[~valid_mask_planar] = np.nan
    xdir_planar[valid_mask_planar] /= ll_planar[valid_mask_planar]

    ydir_planar = np.column_stack((-xdir_planar[:, 1], xdir_planar[:, 0], np.zeros(nf)))
    ydir_planar /= np.linalg.norm(ydir_planar, axis=1, keepdims=True)

    zdir_fixed = np.tile([0, 0, 1], (nf, 1))

    r_root_inv_oriented = np.stack([xdir_planar, ydir_planar, zdir_fixed], axis=1)

    # Compute direction to mid-spine (if available)
    valid_mid_spine = ~np.isnan(spineM_x)
    if np.any(valid_mid_spine):
        dir_to_butt = np.column_stack((spineF_x - spineM_x, spineF_y - spineM_y, spineF_z - spineM_z))
        ll_butt = np.linalg.norm(dir_to_butt, axis=1, keepdims=True)
        valid_butt_mask = (ll_butt[:, 0] >= 0.001)
        dir_to_butt[~valid_butt_mask] = np.nan
        dir_to_butt[valid_butt_mask] /= ll_butt[valid_butt_mask]

        dir_to_butt = np.einsum('ijk,ik->ij', r_root_inv, dir_to_butt)
        dir_backs = dir_to_butt

    return r_roots, r_root_inv_oriented, r_root_inv, dir_backs


In [31]:
import numpy as np
import pandas as pd

# Simulate data: 10 frames with 3D points (neck and tail keypoints)
np.random.seed(0)
test_sorted_point_data = np.random.rand(10, 8, 3)  # 10 frames, 8 keypoints, 3D (x, y, z)

# For the dataframe format
test_data_dict = {
    "kp4_x": test_sorted_point_data[:, 4, 0],
    "kp4_y": test_sorted_point_data[:, 4, 1],
    "kp4_z": test_sorted_point_data[:, 4, 2],
    "kp6_x": test_sorted_point_data[:, 6, 0],
    "kp6_y": test_sorted_point_data[:, 6, 1],
    "kp6_z": test_sorted_point_data[:, 6, 2]
}
test_df = pd.DataFrame(test_data_dict)


def get_body_rotations_original(sorted_point_data):
    nf = len(sorted_point_data)
    r_roots = np.full((nf, 3, 3), np.nan)

    for t in range(nf):
        if np.isnan(sorted_point_data[t, 4, 0]) or np.isnan(sorted_point_data[t, 6, 0]):
            continue

        xdir = sorted_point_data[t, 4, :] - sorted_point_data[t, 6, :]
        xdir[2] = 0.
        ll = np.linalg.norm(xdir)
        if ll < 0.001:
            continue
        xdir = xdir / ll

        zdir = np.array([0, 0, 1])
        ydir = np.cross(zdir, xdir)
        ydir = ydir / np.linalg.norm(ydir)

        r_roots[t, :, :] = np.array([xdir, ydir, zdir]).T

    return r_roots


def get_body_rotations_modified(df):
    nf = df.shape[0]
    r_roots = np.full((nf, 3, 3), np.nan)

    spineF_x, spineF_y, spineF_z = df["kp4_x"], df["kp4_y"], df["kp4_z"]
    tailB_x, tailB_y, tailB_z = df["kp6_x"], df["kp6_y"], df["kp6_z"]

    valid_mask = ~np.isnan(spineF_x) & ~np.isnan(tailB_x)
    xdir = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, np.zeros(nf)))
    ll = np.linalg.norm(xdir, axis=1, keepdims=True)

    valid_mask &= (ll[:, 0] >= 0.001)
    xdir[~valid_mask] = np.nan
    xdir[valid_mask] /= ll[valid_mask]

    zdir = np.array([0, 0, 1])
    ydir = np.cross(zdir, xdir)
    ydir /= np.linalg.norm(ydir, axis=1, keepdims=True)

    r_roots = np.stack([xdir, ydir, np.tile(zdir, (nf, 1))], axis=1)
    r_roots = np.transpose(r_roots, (0, 2, 1))

    return r_roots

# Run both functions
r_roots_original = get_body_rotations_original(test_sorted_point_data)
r_roots_modified = get_body_rotations_modified(test_df)

# Compare results
print("Are the outputs identical?", np.allclose(r_roots_original, r_roots_modified, equal_nan=True))
print("\nOriginal Output (first frame):\n", r_roots_original[0])
print("\nModified Output (first frame):\n", r_roots_modified[0])



Are the outputs identical? True

Original Output (first frame):
 [[-0.96674328 -0.25574876  0.        ]
 [ 0.25574876 -0.96674328  0.        ]
 [ 0.          0.          1.        ]]

Modified Output (first frame):
 [[-0.96674328 -0.25574876  0.        ]
 [ 0.25574876 -0.96674328  0.        ]
 [ 0.          0.          1.        ]]


In [30]:
xdir_no_proj = np.array([
    [0.1, 0.2, 0.0],
    [0.3, 0.4, 0.0],
    [0.5, 0.6, 0.0],
    [0.7, 0.8, 0.0],
    [0.9, 1.0, 0.0]
])  # Shape (5, 3)

print(xdir_no_proj[:, 1])  # Get all y values (second column)
# Output: [0.2 0.4 0.6 0.8 1.0]  # Shape (5,)
print(xdir_no_proj[1])  # Get the entire second row
# Output: [0.3 0.4 0.0]  # Shape (3,)


[0.2 0.4 0.6 0.8 1. ]
[0.3 0.4 0. ]


In [ ]:
def get_body_rotations(df, spineF_kp=4, tailB_kp=6):
    nf = df.shape[0]
    r_roots = np.zeros((nf, 3, 3)) 


    spineF_x, spineF_y = df[f"kp{spineF_kp}_x"], df[f"kp{spineF_kp}_y"]
    tailB_x, tailB_y = df[f"kp{tailB_kp}_x"], df[f"kp{tailB_kp}_y"]
    # Precompute valid mask
    valid_mask = ~np.isnan(spineF_x) & ~np.isnan(tailB_x)
    xdir = np.column_stack((spineF_x - tailB_x, spineF_y - tailB_y, np.zeros_like(spineF_z)))  # z=0 projection

    # xdir = np.column_stack([
    #     df[f"kp{spineF_kp}_x"] - df[f"kp{tailB_kp}_x"],
    #     df[f"kp{spineF_kp}_y"] - df[f"kp{tailB_kp}_y"],
    #     np.zeros(nf)  # Set z-component to 0 for projection onto the XY plane
    # ])
    ll = np.linalg.norm(xdir, axis=1, keepdims=True) 
        # if ll < 0.001:
        #     continue
    # xdir = xdir / ll
    valid_mask &= (ll[:, 0] >= 0.001)  # Ensure non-trivial length
    xdir[~valid_mask] = np.nan  # Assign NaN to invalid frames
    xdir[valid_mask] /= ll[valid_mask]  # Normalize

    zdir = np.array([0, 0, 1])
    ydir = np.cross(zdir, xdir) 
    # Construct transformation matrices
    shitinv = np.stack([xdir, ydir, np.tile(zdir, (nf, 1))], axis=1)  # (frames, 3, 3)
    # shitinv = (np.array([xdir, ydir, zdir])) + 0.  # mapping from global coordinates to animal coordinates
       
    # Transpose to get r_roots mapping from animal to global
    r_roots = np.transpose(shitinv, (0, 2, 1))




In [29]:
import numpy as np
import pandas as pd

# Mock Data: 5 frames of keypoints (neck = kp4, tail = kp6)
data = {
    "kp4_x": [1, 2, 3, np.nan, 5],  # Neck x-coordinates
    "kp4_y": [1, 2, 3, 4, 5],        # Neck y-coordinates
    "kp4_z": [0, 0, 0, 0, 0],        # Neck z-coordinates
    "kp6_x": [0, 1, 2, 3, np.nan],   # Tail x-coordinates
    "kp6_y": [0, 1, 2, 3, 4],        # Tail y-coordinates
    "kp6_z": [0, 0, 0, 0, 0]         # Tail z-coordinates
}
df_merged_with_dF_F = pd.DataFrame(data)

# Step 1: Extract keypoints as NumPy arrays
neck_x, neck_y, neck_z = df_merged_with_dF_F["kp4_x"].to_numpy(), df_merged_with_dF_F["kp4_y"].to_numpy(), df_merged_with_dF_F["kp4_z"].to_numpy()
tail_x, tail_y, tail_z = df_merged_with_dF_F["kp6_x"].to_numpy(), df_merged_with_dF_F["kp6_y"].to_numpy(), df_merged_with_dF_F["kp6_z"].to_numpy()

# Step 2: Compute direction vectors
valid_mask = ~np.isnan(neck_x) & ~np.isnan(tail_x)
xdir = np.column_stack((neck_x - tail_x, neck_y - tail_y, np.zeros_like(neck_z)))  # Ignore z-axis

# Normalize xdir
ll = np.linalg.norm(xdir, axis=1, keepdims=True)
valid_mask &= (ll[:, 0] >= 0.001)  # Ensure non-trivial length
xdir[~valid_mask] = np.nan  # Assign NaN to invalid frames
xdir[valid_mask] /= ll[valid_mask]  # Normalize

# Compute ydir using right-hand rule
zdir = np.array([0, 0, 1])  # Fixed global z-direction
ydir = np.cross(zdir, xdir)

# ---- Original shitinv (looped) ----
original_shitinv = []
for t in range(len(df_merged_with_dF_F)):
    if np.isnan(neck_x[t]) or np.isnan(tail_x[t]):
        original_shitinv.append(np.full((3, 3), np.nan))  # Skip invalid frames
        continue
    
    xdir_t = np.array([neck_x[t] - tail_x[t], neck_y[t] - tail_y[t], 0])
    ll_t = np.linalg.norm(xdir_t)
    
    if ll_t < 0.001:
        original_shitinv.append(np.full((3, 3), np.nan))
        continue
    
    xdir_t /= ll_t
    ydir_t = np.cross([0, 0, 1], xdir_t)
    zdir_t = np.array([0, 0, 1])

    shitinv_t = np.array([xdir_t, ydir_t, zdir_t])  # From global to animal
    original_shitinv.append(np.transpose(shitinv_t))  # From animal to global

original_shitinv = np.array(original_shitinv)

# ---- New shitinv (vectorized) ----
new_shitinv = np.stack([xdir, ydir, np.tile(zdir, (len(df_merged_with_dF_F), 1))], axis=1)
new_shitinv = np.transpose(new_shitinv, (0, 2, 1))  # From animal to global

# ---- Print Results ----
print("Original shitinv (loop-based):\n", original_shitinv)
print("\nNew shitinv (vectorized):\n", new_shitinv)


Original shitinv (loop-based):
 [[[ 0.70710678 -0.70710678  0.        ]
  [ 0.70710678  0.70710678  0.        ]
  [ 0.          0.          1.        ]]

 [[ 0.70710678 -0.70710678  0.        ]
  [ 0.70710678  0.70710678  0.        ]
  [ 0.          0.          1.        ]]

 [[ 0.70710678 -0.70710678  0.        ]
  [ 0.70710678  0.70710678  0.        ]
  [ 0.          0.          1.        ]]

 [[        nan         nan         nan]
  [        nan         nan         nan]
  [        nan         nan         nan]]

 [[        nan         nan         nan]
  [        nan         nan         nan]
  [        nan         nan         nan]]]

New shitinv (vectorized):
 [[[ 0.70710678 -0.70710678  0.        ]
  [ 0.70710678  0.70710678  0.        ]
  [ 0.          0.          1.        ]]

 [[ 0.70710678 -0.70710678  0.        ]
  [ 0.70710678  0.70710678  0.        ]
  [ 0.          0.          1.        ]]

 [[ 0.70710678 -0.70710678  0.        ]
  [ 0.70710678  0.70710678  0.        ]
  [ 0.

In [26]:
xdir = np.array([[3, 4, 0],  # Frame 1
                 [4, 3, 0],  # Frame 2
                 [5, 12, 0]])  # Frame 3

norm_xy = np.linalg.norm(xdir)  # ❌ Incorrect! No axis specified.
print(norm_xy)  # Outputs a **single** scalar value (the norm of the entire matrix)


14.798648586948742


In [27]:
norm_xy = np.linalg.norm(xdir, axis=1)  # ✅ Correct! Computes row-wise norms.
print(norm_xy)  # Outputs an array: [5. 5. 13.]


[ 5.  5. 13.]


In [25]:
def get_body_rotations(df, spineF_kp=4, tailB_kp=6):
    # xdir_x = df["kp{spineF_kp}_x"] - df[f"kp{tailB_kp}_x"]
    # xdir_y = df["kp{spineF_kp}_y"] - df[f"kp{tailB_kp}_y"]
    # xdir_z = df["kp{spineF_kp}_z"] - df[f"kp{tailB_kp}_z"]\
    nf = df.shape[0]
    r_roots = np.zeros((nf, 3, 3))  #

    # axes = ['x', 'y', 'z']
    # xdir = {axis: df[f"kp{spineF_kp}_{axis}"] - df[f"kp{tailB_kp}_{axis}"] for axis in axes}
    # xdir = {
    #     axis: df[f"kp{spineF_kp}_{axis}"] - df[f"kp{tailB_kp}_{axis}"]
    #     for axis in ['x', 'y', 'z']
    # }
    # xdir_x, xdir_y, xdir_z = xdir['x'], xdir['y'], xdir['z']
    

    # Norm in the XY plane (shape: one value per row)
    # norm_xy = np.sqrt(xdir_x**2 + xdir_y**2)
    # if norm_xy < 0.001:
    #     continue

    # xdir_x, xdir_y = xdir_x/norm_xy, xdir_y/norm_xy

    # zdir_1 = np.ravel(np.array([0, 0, 1]))

    # ydir_cross = np.cross(zdir_1, xdir)

    # shitinv = (np.array([xdir, ydir_cross, zdir_1])) + 0.  # mapping from global coordinates to animal coordinates
    # # r_roots[t, :, :] = np.transpose(shitinv)  # mapping from animal to global from the top view


whw = get_body_rotations(data_load)

(8894,)


AxisError: axisb: axis -1 is out of bounds for array of dimension 0

In [23]:
def get_body_rotations(df, spineF_kp=4, tailB_kp=6):
    print('Processing to get body related rotation matrix ...... ')
    # nf = len(sorted_point_data)  # number of frames
    nf = sorted_point_data.shape[0]
    r_roots = np.zeros((nf, 3, 3))  #
    r_root_inv = np.zeros((nf, 3, 3))  #
    r_root_inv_oriented = np.zeros((nf, 3, 3))  #
    dir_backs = np.zeros((nf, 3))  #
    r_roots[:] = np.nan
    r_root_inv[:] = np.nan
    r_root_inv_oriented[:] = np.nan
    dir_backs[:] = np.nan

    timeofheadnans = []
    # DO IT ALL FOR THE NEW ROOT NODE (ASSUMING BUTT TO NECK!)

    # Extract neck and tail coordinates dynamically
    # neck_coords = sorted_point_data[[f"kp{spineF_kp}_{axis}" for axis in ['x', 'y', 'z']]].to_numpy()
    # tail_coords = sorted_point_data[[f"kp{tailB_kp}_{axis}" for axis in ['x', 'y', 'z']]].to_numpy()

    
    # neck_x, neck_y, neck_z = sorted_point_data[f"kp{spineF_kp}_x"], sorted_point_data[f"kp{spineF_kp}_y"], sorted_point_data[f"kp{spineF_kp}_z"]
    # tail_x, tail_y, tail_z = sorted_point_data[f"kp{tailB_kp}_x"], sorted_point_data[f"kp{tailB_kp}_y"], sorted_point_data[f"kp{tailB_kp}_z"]

    # xdir_x = neck_x - tail_x
    # xdir_y = neck_y - tail_y
    # xdir_z = 0  # For 2D projection

    xdir_x = df["kp{spineF_kp}_x"] - df[f"kp{tailB_kp}_x"]
    xdir_y = df["kp{spineF_kp}_y"] - df[f"kp{tailB_kp}_y"]

    for t in range(len(sorted_point_data)):
        if np.isnan(neck_coords[t]).any() or np.isnan(tail_coords[t]).any():
            continue
        

        # Compute xdir (top-view direction from tail to neck)
        # xdir = neck_coords - tail_coords
        # xdir[:, 2] = 0  # Project onto the XY plane
        # xdir_norm = np.linalg.norm(xdir, axis=1, keepdims=True)
        # xdir = np.divide(xdir, xdir_norm, out=np.zeros_like(xdir), where=xdir_norm != 0)  # Normalize
    
        # THIS IS A STUPID WAY OF CREATING RROOTS (ANGLE OF BODY RELATIVE TO A WALL)
        # It should just be xdir with z=0...
        xdir = neck_coords[t] - tail_coords[t]  # from tail to neck
        xdir[2] = 0.0
        ll = np.linalg.norm(xdir)
        if ll < 0.001:
            continue
        xdir = xdir / ll  # x direction of the animal from the top view (chosen)
        zdir = np.ravel(np.array([0, 0, 1]))
        ydir = np.cross(zdir, xdir)  # y direction according to right-hand rule
        shitinv = (np.array([xdir, ydir, zdir])) + 0.  # mapping from global coordinates to animal coordinates
        r_roots[t, :, :] = np.transpose(shitinv)  # mapping from animal to global from the top view
    
        # HERE IS THE REAL ROOT ANGLE!!!
        # SAME AS WITHLOCKINZ = TRUE AND WITHROTATIONS = TRUE
        xdir = neck_coords[t] - tail_coords[t]
        ll = np.linalg.norm(xdir)
        if ll < 0.001:
            continue
        xdir = xdir / ll  # unit vector from tail to neck in global coordinates (not top view)
        ydir = np.zeros(3)
        ydir[0] = -xdir[1] + 0.
        ydir[1] = xdir[0] + 0.
        ydir[2] = 0.
        ydir = ydir / np.linalg.norm(ydir)  # an orthogonal vector to the x dir in xy plane
        zdir = np.cross(xdir, ydir)
        zdir = zdir / np.linalg.norm(zdir)
        r_root_inv[t, :, :] = (np.array([xdir, ydir, zdir])) + 0.  # for ego3
    
        # HERE IS THE OTHER OTHER OTHER ROOT ANGLE!!!
        xdir = neck_coords[t] - tail_coords[t]
        xdir[2] = 0  # so we just keep the planar direction from the body coordinates!!!
        ll = np.linalg.norm(xdir)
        if ll < 0.001:
            timeofheadnans.append(t)
            continue
        xdir = xdir / ll
        ydir = np.zeros(3)
        ydir[0] = -xdir[1] + 0.
        ydir[1] = xdir[0] + 0.
        ydir[2] = 0.
        ydir = ydir / np.linalg.norm(ydir)
        zdir = np.zeros(3)
        zdir[2] = 1.
        r_root_inv_oriented[t, :, :] = (np.array([xdir, ydir, zdir])) + 0.  # for ego2
    
        # if( WITHLOCKEDINZ == False and WITHROTATIONS == True ):
        # xdir = sorted_point_data[t,4,:] - sorted_point_data[t,6,:]
        # ll = np.linalg.norm(xdir)
        # if(ll<0.001):
        # continue
        # xdir[2] = 0.
        # xdir = xdir / ll
        # ydir = zeros(3)
        # ydir[0] = -xdir[1]+0.
        # ydir[1] = xdir[0]+0.
        # ydir[2] = 0.
        # ydir = ydir/np.linalg.norm(ydir)
        # zdir = zeros(3)
        # zdir[2] = 1.
        # RrootINV = ( np.array([xdir, ydir, zdir]) ) + 0.
        if ~np.isnan(tail_coords[t]):
            dir_to_butt = neck_coords[t] - tail_coords[t]
            dir_to_butt = dir_to_butt / np.linalg.norm(dir_to_butt)
            dir_to_butt = np.dot(r_root_inv[t, :, :], dir_to_butt)
            dir_backs[t, :] = dir_to_butt + 0.
    
    return r_roots, r_root_inv_oriented, r_root_inv, dir_backs


def get_head_rotations(head_x, head_z, r_root_inv, r_root_inv_oriented):
    print('Processing to get head related rotation matrix ...... ')
    nf = len(head_x)  # number of frames
    
    global_head_rm = np.zeros((nf, 3, 3))  # global head rotation matrix
    r_heads = np.zeros((nf, 3, 3))  #
    head_ups = np.zeros(nf)  #
    body_turned_heads = np.zeros((nf, 3, 3))  #
    
    body_turned_heads[:] = np.nan
    head_ups[:] = np.nan
    global_head_rm[:] = np.nan
    r_heads[:] = np.nan

    for t in np.arange(nf):
        if (r_root_inv is not None) and (r_root_inv_oriented is not None):
            if np.isnan(r_root_inv[t, 0, 0]):
                continue
        else:
            print('Only global head rotation matrix is generated.')

        if ~np.isnan(head_x[t, 0]):
            hx = head_x[t] / np.linalg.norm(head_x[t])
            hz = head_z[t] / np.linalg.norm(head_z[t])
            hy = np.cross(hz, hx)
            global_head_rm[t, :, :] = np.array([hx, hy, hz])  # global to head

            if (r_root_inv is not None) and (r_root_inv_oriented is not None):
                rhx = np.dot(r_root_inv[t, :, :], hx)
                rhz = np.dot(r_root_inv[t, :, :], hz)
                rhy = np.dot(r_root_inv[t, :, :], hy)
                r_heads[t, :, :] = np.array([rhx, rhy, rhz])  # ego3_head_rotm
                # hy = head_y[t]/np.linalg.norm(head_y[t])  ## these were screwed up at in head csys maker of gui (oops)
                # rhy = dot(RrootINV, hy)
                
                rhx2 = np.dot(r_root_inv_oriented[t, :, :], hx)
                rhz2 = np.dot(r_root_inv_oriented[t, :, :], hz)
                rhy2 = np.dot(r_root_inv_oriented[t, :, :], hy)
                body_turned_heads[t, :, :] = np.array([rhx2, rhy2, rhz2])  # ego2_head_rotm

            head_ups[t] = np.dot(hx, [0, 0, 1])
        
    return global_head_rm, r_heads, body_turned_heads, head_ups

whw = get_body_rotations(data_load)
# wheee = get_head_rotations(data_load['kp3_x'],data_load['kp3_z'])

Processing to get body related rotation matrix ...... 
[[ 30.71986961 536.19567871  17.56861115]
 [ 30.22535515 536.33056641  17.10687637]
 [ 29.23507881 538.16186523  17.5914135 ]
 ...
 [583.64404297 508.06555176  19.45136452]
 [587.91369629 513.77038574  20.0354557 ]
 [591.6607666  515.85565186  22.5261116 ]]
[[ 6.76125183e+01  5.01296997e+02 -4.46997613e-01]
 [ 6.78170471e+01  5.00634247e+02 -1.25118211e-01]
 [ 6.74209747e+01  5.00797974e+02 -2.05259025e-01]
 ...
 [ 5.44862183e+02  4.71971802e+02  6.03607988e+00]
 [ 5.58674377e+02  4.74017181e+02  5.65276575e+00]
 [ 5.57506958e+02  4.85555756e+02  8.19240379e+00]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

(8894, 245)